# OLS, The Frisch-Waugh Theorem

This notebook illustrates the Frish-Waugh theorem.

In particular, it shows the following. First, we regress

$y = x_1'\beta_1 + x_2'\beta_2 + u$

Second, we run three regressions

1. $y = x_1'\gamma_1 + e_y$

2. $x_2 = x_1'\delta + e_2$.

3. $e_y = e_2'\theta + v$,
where $(e_y,e_2)$ are from the regressions in 1. and 2.

Then, the estimates of $\beta_2$ and $\theta$ will be the same (as will their standard errors).


## Load Packages and Extra Functions

In [1]:
using Printf, DelimitedFiles, LinearAlgebra, Distributions

include("jlFiles/printmat.jl")
include("jlFiles/Ols.jl");         #functions for OLS

## Loading Data

In [2]:
x = readdlm("Data/FFmFactorsPs.csv",',',skipstart=1)

y  = x[:,2]/100
x₁ = [ones(length(y)) x[:,3]/100]             #1st set of regressors (2)
x₂ = x[:,4]/100                               #2nd set of regressors (here just 1)
x = nothing

## Regress y on Both x₁ and x₂

In [3]:
(b,_,_,V,) = OlsGMFn(y,[x₁ x₂])
std_b = sqrt.(diag(V))

printblue("OLS Results from y regressed on x:\n")
printmat([b std_b],colNames=["b","std"],rowNames=["x₁ (1)","x₁ (2)","x₂"])

println("Remark: x₁ (1) indicates variable 1 in x₁")

OLS Results from y regressed on x:

               b       std
x₁ (1)     0.007     0.002
x₁ (2)     0.217     0.073
x₂        -0.429     0.074

Remark: x₁ (1) indicates variable 1 in x₁


## The Three Steps in Frisch-Waugh

1. Regress y on x₁ and save the residuals as e_y. (Sorry, cannot create a symbol like $e_y$.)

2. Regress x₂ on x₁ and save the residuals as e₂.

3. Regress e_y on e₂.

In [4]:

(_,e_y,) = OlsGMFn(y,x₁);              #step 1

(_,e₂,) = OlsGMFn(x₂,x₁);              #step 2

(b,_,_,V,) = OlsGMFn(e_y,e₂)           #step 3
std_b = isa(V,Number) ? sqrt(V) : sqrt.(diag(V)) #diag() fails if V is a number (not a matrix)

printblue("OLS Results from e_y regressed on e₂:\n")
printmat([b std_b],colNames=["b","std"],rowNames=["e₂"])
printred("Should be same coeff and std as in multiple regression (above)")

OLS Results from e_y regressed on e₂:

           b       std
e₂    -0.429     0.074

Should be same coeff and std as in multiple regression (above)
